## 3.2 운영 중인 노드에 실행 명령하기

In [1]:
import requests
import json
import pandas as pd
import hashlib
import random

### 1. 노드의 블록 정보 확인

In [2]:
# one_node.ipynb 실행켜야 서버에서 정보 받아옴

headers = {'Content-Type': 'application/json; charset=utf-8'}
res = requests.get("http://localhost:5000/chain", headers=headers)
json.loads(res.content)

{'chain': [{'index': 1,
   'nonce': 100,
   'previous_hash': 1,
   'timestamp': 1726464312.4814615,
   'transactions': []}],
 'length': 1}

### 2-1. transaction 입력하기

In [3]:
# transaction 입력하기
headers = {'Content-Type' : 'application/json; charset=utf-8'}
data = {
    "sender": "test-from",
    "recipient": "test-to",
    "amount" : 3,
}

requests.post("http://localhost:5000/transactions/new", headers=headers,data = json.dumps(data)).content

b'{"message":"Transaction will be added to Block {2}"}\n'

### 2-2. 노드의 블록 정보 확인 - 2

In [4]:
headers = {'Content-Type': 'application/json; charset=utf-8'}
res = requests.get("http://localhost:5000/chain", headers=headers)
json.loads(res.content)

{'chain': [{'index': 1,
   'nonce': 100,
   'previous_hash': 1,
   'timestamp': 1726464312.4814615,
   'transactions': []}],
 'length': 1}

### 3-1. 채굴하기 -1

In [5]:
# 채굴하기
headers = {'Content-Type' : 'application/json; charset=utf-8'}
res = requests.get("http://localhost:5000/mine")
print(res)

<Response [200]>


### 3-2. 노드의 블록 정보 확인 -3

In [6]:
headers = {'Content-Type': 'application/json; charset=utf-8'}
res = requests.get("http://localhost:5000/chain", headers=headers)
json.loads(res.content)

{'chain': [{'index': 1,
   'nonce': 100,
   'previous_hash': 1,
   'timestamp': 1726464312.4814615,
   'transactions': []},
  {'index': 2,
   'nonce': 3360337,
   'previous_hash': 'a836d9a7c2b5097617d38a43956e6a999cdf20405f7507caad2f3bf115c84e8f',
   'timestamp': 1726464325.365661,
   'transactions': [{'amount': 3,
     'recipient': 'test-to',
     'sender': 'test-from',
     'timestamp': 1726464321.2126505},
    {'amount': 0.1,
     'recipient': 'node_5000',
     'sender': 'master',
     'timestamp': 1726464325.365661}]}],
 'length': 2}

### 4. transaction2 입력하기 + 채굴 + 블록 정보 확인

In [7]:
# transaction 입력하기 - 2
headers = {'Content-Type' : 'application/json; charset=utf-8'}
data = {
    "sender": "test-from",
    "recipient": "test-to2",
    "amount" : 30,
}

requests.post("http://localhost:5000/transactions/new", headers=headers,data = json.dumps(data)).content

# transaction 입력하기 - 3
headers = {'Content-Type' : 'application/json; charset=utf-8'}
data = {
    "sender": "test-from",
    "recipient": "test-to3",
    "amount" : 300,
}

requests.post("http://localhost:5000/transactions/new", headers=headers,data = json.dumps(data)).content

# 채굴하기
headers = {'Content-Type' : 'application/json; charset=utf-8'}
res = requests.get("http://localhost:5000/mine")
print(res)

# 노드의 블록정보 확인 - 4
headers = {'Content-Type': 'application/json; charset=utf-8'}
res = requests.get("http://localhost:5000/chain", headers=headers)

<Response [200]>


In [8]:
res.text

'{"chain":[{"index":1,"nonce":100,"previous_hash":1,"timestamp":1726464312.4814615,"transactions":[]},{"index":2,"nonce":3360337,"previous_hash":"a836d9a7c2b5097617d38a43956e6a999cdf20405f7507caad2f3bf115c84e8f","timestamp":1726464325.365661,"transactions":[{"amount":3,"recipient":"test-to","sender":"test-from","timestamp":1726464321.2126505},{"amount":0.1,"recipient":"node_5000","sender":"master","timestamp":1726464325.365661}]},{"index":3,"nonce":7941146,"previous_hash":"cef6b241be14fab4256bed76c51d52a3a6835484d22ab31a3164ab4153d5ffe4","timestamp":1726464333.5811384,"transactions":[{"amount":30,"recipient":"test-to2","sender":"test-from","timestamp":1726464329.4395347},{"amount":300,"recipient":"test-to3","sender":"test-from","timestamp":1726464331.5102217},{"amount":0.1,"recipient":"node_5000","sender":"master","timestamp":1726464333.5811384}]}],"length":3}\n'

### 5. Pandas를 활용한 거래내역 조회

In [9]:
status_json = json.loads(res.text)
status_json['chain']
tx_amount_l = []
tx_sender_l = []
tx_reciv_l = []
tx_time_l = []

for chain_index in range(len(status_json['chain'])):
    chain_tx = status_json['chain'][chain_index]['transactions']
    for each_tx in range(len(chain_tx)):
        tx_amount_l.append(chain_tx[each_tx]['amount'])
        tx_sender_l.append(chain_tx[each_tx]['sender'])
        tx_reciv_l.append(chain_tx[each_tx]['recipient'])
        tx_time_l.append(chain_tx[each_tx]['timestamp'])

df_tx = pd.DataFrame()
df_tx['timestamp'] = tx_time_l
df_tx['sender'] = tx_sender_l
df_tx['recipient'] = tx_reciv_l
df_tx['amount'] = tx_amount_l
df_tx

,timestamp,sender,recipient,amount
0,1.726464e+09,test-from,test-to,3.0
1,1.726464e+09,master,node_5000,0.1
2,1.726464e+09,test-from,test-to2,30.0
3,1.726464e+09,test-from,test-to3,300.0
4,1.726464e+09,master,node_5000,0.1


### 6. 거래내역 기반 계정별(지갑별) 잔액 조회

In [10]:
df_sended = pd.DataFrame(df_tx.groupby('sender')['amount'].sum()).reset_index()
df_sended.columns = ['user', 'sended_amount']
df_received = pd.DataFrame(df_tx.groupby('recipient')['amount'].sum()).reset_index()
df_received.columns = ['user', 'received_amount']
df_received

df_status = pd.merge(df_received, df_sended, on = 'user', how = 'outer').fillna(0)
df_status['balance'] = df_status['received_amount'] - df_status['sended_amount']
df_status

,user,received_amount,sended_amount,balance
0,master,0.0,0.2,-0.2
1,node_5000,0.2,0.0,0.2
2,test-from,0.0,333.0,-333.0
3,test-to,3.0,0.0,3.0
4,test-to2,30.0,0.0,30.0
5,test-to3,300.0,0.0,300.0
